In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import os
import glob

In [2]:
def score(df, key, score_name, negative=True):
    mu = df[key].mean()
    s = df[key].std()
    
    if negative:
        df[score_name] = ((-(df[key] - mu) / s) * 10 + 50).round(1)
    else:
        df[score_name] = (((df[key] - mu) / s) * 10 + 50).round(1)

In [3]:
basepath = os.path.expanduser('~/Documents/programming-projects/nfl-big-data/calculated-data')

In [4]:
glob.glob(os.path.join(basepath,'*.csv'))

['/Users/jacobhow/Documents/programming-projects/nfl-big-data/calculated-data/man play results.csv',
 '/Users/jacobhow/Documents/programming-projects/nfl-big-data/calculated-data/player max speed + accel.csv',
 '/Users/jacobhow/Documents/programming-projects/nfl-big-data/calculated-data/man coverage production reduction.csv',
 '/Users/jacobhow/Documents/programming-projects/nfl-big-data/calculated-data/offensive production.csv',
 '/Users/jacobhow/Documents/programming-projects/nfl-big-data/calculated-data/player season coverages.csv',
 '/Users/jacobhow/Documents/programming-projects/nfl-big-data/calculated-data/team target coverage epa analysis.csv',
 '/Users/jacobhow/Documents/programming-projects/nfl-big-data/calculated-data/team personal epa analysis.csv',
 '/Users/jacobhow/Documents/programming-projects/nfl-big-data/calculated-data/non targeted man coverage epa analysis.csv',
 '/Users/jacobhow/Documents/programming-projects/nfl-big-data/calculated-data/coverage scheme epa analysis.

## Player Level

In [5]:
player_season_coverages = pd.read_csv(os.path.join(basepath,'player season coverages.csv'),index_col=0)

In [6]:
man_non_targeted_epa_analysis = pd.read_csv(os.path.join(basepath,'non targeted man coverage epa analysis.csv'),index_col=0)

In [7]:
man_play_results = pd.read_csv(os.path.join(basepath,'man play results.csv'),index_col=0)

In [8]:
player_season_coverages.head()

,displayName,position,snaps,zone,zone-deep,zone-over,man,man-over,blitz,games played,Team,zone %,zone-deep %,zone-over %,man %,man-over %,blitz %,zone-total,man-total
306,Dominique Rodgers-Cromartie,CB,64,7,17,25,3,12,0,5,OAK,10.9,26.6,39.1,4.7,18.8,0.0,49,15
1302,Aqib Talib,CB,202,19,79,60,25,19,0,8,LA,9.4,39.1,29.7,12.4,9.4,0.0,158,44
2307,Orlando Scandrick,CB,462,43,89,84,146,100,0,13,KC,9.3,19.3,18.2,31.6,21.6,0.0,216,246
2354,Wesley Woodyard,ILB,381,54,12,250,0,5,60,14,TEN,14.2,3.1,65.6,0.0,1.3,15.7,316,5
4365,Brandon Carr,CB,517,69,91,98,154,94,11,16,BAL,13.3,17.6,19.0,29.8,18.2,2.1,258,248


In [9]:
man_non_targeted_epa_analysis.head()

,non targets,non targets epa,non targets - iso,non targets epa - iso,non targets - over,non targets epa - over
2557958,46,2.792,30,1.738,16,1.054
2555383,53,2.358,48,2.130,5,0.228
79848,83,3.432,66,2.770,17,0.662
2534832,1,0.037,1,0.037,0,0.000
2556363,9,0.269,6,0.103,3,0.166


In [10]:
man_play_results.head()

,db nflId,db name,rc nflId,rc name,play epa,expected epa,epa delta,safety help
4406,2560851,Justin Reid,2540160,Jordan Reed,-11.372110,-0.140,-11.232110,True
931,71251,Patrick Chung,2495139,Charles Clay,-9.994132,-0.498,-9.496132,True
2162,2558982,Hardy Nickerson,2540232,Jack Doyle,-9.325884,-0.125,-9.200884,True
4684,2550257,Daniel Sorensen,71265,Jared Cook,-9.144098,0.227,-9.371098,False
5038,2559164,Michael Davis,2558856,David Moore,-8.794337,0.254,-9.048337,False


In [11]:
columns = ['id','name','targeted man coverage count','man delta epa total','man delta epa rate',
           'targeted man-iso coverage count','man-iso delta epa total','man-iso delta epa rate',
           'targeted man-over coverage count','man-over delta epa total','man-over delta epa rate',
           'targeted - safety help %']

In [12]:
totals = []

for _id in man_play_results['db nflId'].unique():
    db_df = man_play_results[man_play_results['db nflId'] == _id]
    db_name = db_df['db name'].values[0]

    nTotalPlays = db_df.shape[0]

    db_epa_total = db_df['epa delta'].sum()
    db_epa_rate = db_df['epa delta'].mean()
    
    iso_df = db_df[db_df['safety help'] == False]
    over_df = db_df[db_df['safety help'] == True]
    
    nIsoPlays = iso_df.shape[0]
    
    db_iso_epa_total = iso_df['epa delta'].sum()
    db_iso_epa_rate = iso_df['epa delta'].mean()
    
    nOverPlays = over_df.shape[0]
    
    db_over_epa_total = over_df['epa delta'].sum()
    db_over_epa_rate = over_df['epa delta'].mean()

    safety_help_percent = round((db_df['safety help'].astype(int).sum() / db_df.shape[0]) * 100, 1) 

    totals.append([_id,db_name, nTotalPlays, db_epa_total, db_epa_rate,
                   nIsoPlays, db_iso_epa_total, db_iso_epa_rate,
                   nOverPlays, db_epa_total, db_epa_rate,
                   safety_help_percent])
    
totals = pd.DataFrame(totals,columns=columns).set_index('id')

In [13]:
totals.head()

,name,targeted man coverage count,man delta epa total,man delta epa rate,targeted man-iso coverage count,man-iso delta epa total,man-iso delta epa rate,targeted man-over coverage count,man-over delta epa total,man-over delta epa rate,targeted - safety help %
id,,,,,,,,,,,
2560851,Justin Reid,2,-13.171957,-6.585979,1,-1.939847,-1.939847,1,-13.171957,-6.585979,50.0
71251,Patrick Chung,35,1.050480,0.030014,17,6.613346,0.389020,18,1.050480,0.030014,51.4
2558982,Hardy Nickerson,6,-7.497456,-1.249576,2,0.929846,0.464923,4,-7.497456,-1.249576,66.7
2550257,Daniel Sorensen,9,-0.014437,-0.001604,5,-3.708173,-0.741635,4,-0.014437,-0.001604,44.4
2559164,Michael Davis,41,-18.144583,-0.442551,33,-13.576467,-0.411408,8,-18.144583,-0.442551,19.5


### Man Analysis

In [14]:
df = pd.merge(player_season_coverages[['displayName','position','snaps','man-total','man','man-over']],
              totals.drop(columns=['name']),
              left_index=True,right_index=True)

In [15]:
df = pd.merge(df, man_non_targeted_epa_analysis,left_index=True,right_index=True)

In [16]:
df

,displayName,position,snaps,man-total,man,man-over,targeted man coverage count,man delta epa total,man delta epa rate,targeted man-iso coverage count,...,targeted man-over coverage count,man-over delta epa total,man-over delta epa rate,targeted - safety help %,non targets,non targets epa,non targets - iso,non targets epa - iso,non targets - over,non targets epa - over
306,Dominique Rodgers-Cromartie,CB,64,15,3,12,2,0.442579,0.221290,0,...,2,0.442579,0.221290,100.0,13,0.896,3,0.106,10,0.790
1302,Aqib Talib,CB,202,44,25,19,10,-1.184709,-0.118471,7,...,3,-1.184709,-0.118471,30.0,33,1.225,18,0.792,15,0.433
2307,Orlando Scandrick,CB,462,246,146,100,60,8.473536,0.141226,38,...,22,8.473536,0.141226,36.7,180,5.947,106,3.285,74,2.662
4365,Brandon Carr,CB,517,248,154,94,54,-3.716949,-0.068832,34,...,20,-3.716949,-0.068832,37.0,181,8.373,112,4.093,69,4.280
71197,Connor Barwin,OLB,150,8,5,3,1,-0.278214,-0.278214,0,...,1,-0.278214,-0.278214,100.0,7,0.140,5,0.132,2,0.008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2561318,Tae Davis,ILB,190,5,5,0,2,2.831041,1.415520,2,...,0,2.831041,1.415520,0.0,3,0.000,3,0.000,0,0.000
2561340,Tre Herndon,CB,38,25,17,8,5,1.205910,0.241182,3,...,2,1.205910,0.241182,40.0,19,0.386,14,0.263,5,0.123
2561417,Charvarius Ward,CB,67,34,30,4,10,0.674194,0.067419,9,...,1,0.674194,0.067419,10.0,22,1.406,19,1.406,3,0.000
2561527,Mike Ford,CB,198,83,54,29,15,11.249367,0.749958,10,...,5,11.249367,0.749958,33.3,66,3.058,42,1.284,24,1.774


In [17]:
snap_threshold = 200

In [18]:
df = df[df['snaps'] > snap_threshold]
df

,displayName,position,snaps,man-total,man,man-over,targeted man coverage count,man delta epa total,man delta epa rate,targeted man-iso coverage count,...,targeted man-over coverage count,man-over delta epa total,man-over delta epa rate,targeted - safety help %,non targets,non targets epa,non targets - iso,non targets epa - iso,non targets - over,non targets epa - over
1302,Aqib Talib,CB,202,44,25,19,10,-1.184709,-0.118471,7,...,3,-1.184709,-0.118471,30.0,33,1.225,18,0.792,15,0.433
2307,Orlando Scandrick,CB,462,246,146,100,60,8.473536,0.141226,38,...,22,8.473536,0.141226,36.7,180,5.947,106,3.285,74,2.662
4365,Brandon Carr,CB,517,248,154,94,54,-3.716949,-0.068832,34,...,20,-3.716949,-0.068832,37.0,181,8.373,112,4.093,69,4.280
71251,Patrick Chung,SS,525,168,100,68,35,1.050480,0.030014,17,...,18,1.050480,0.030014,51.4,128,3.448,80,2.172,48,1.276
71423,Brian Orakpo,OLB,303,3,2,1,1,-0.343774,-0.343774,0,...,1,-0.343774,-0.343774,100.0,2,0.033,2,0.033,0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2560966,Jerome Baker,LB,343,34,10,24,3,1.639706,0.546569,1,...,2,1.639706,0.546569,66.7,31,0.728,9,0.224,22,0.504
2561001,Grant Haley,CB,245,124,51,73,25,11.187032,0.447481,10,...,15,11.187032,0.447481,60.0,95,4.345,40,1.607,55,2.738
2561015,Lorenzo Carter,OLB,277,12,7,5,2,-2.469402,-1.234701,1,...,1,-2.469402,-1.234701,50.0,10,0.320,6,0.073,4,0.247
2561039,Carlton Davis,CB,362,219,158,61,43,-7.638645,-0.177643,29,...,14,-7.638645,-0.177643,32.6,169,5.541,122,3.842,47,1.699


In [19]:
df['safety help %'] = ((df['man-over'] / df['man-total']) * 100).round(1)
df['man target %'] = ((df['targeted man coverage count'] / df['man-total']) * 100).round(1)
df['man-iso target %'] = ((df['targeted man-iso coverage count'] / df['man']) * 100).round(1)
df['man-over target %'] = ((df['targeted man-over coverage count'] / df['man-over']) * 100).round(1)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [20]:
man_snap_threshold = 100

In [21]:
man_df = df[df['man-total'] > man_snap_threshold]
man_df

,displayName,position,snaps,man-total,man,man-over,targeted man coverage count,man delta epa total,man delta epa rate,targeted man-iso coverage count,...,non targets,non targets epa,non targets - iso,non targets epa - iso,non targets - over,non targets epa - over,safety help %,man target %,man-iso target %,man-over target %
2307,Orlando Scandrick,CB,462,246,146,100,60,8.473536,0.141226,38,...,180,5.947,106,3.285,74,2.662,40.7,24.4,26.0,22.0
4365,Brandon Carr,CB,517,248,154,94,54,-3.716949,-0.068832,34,...,181,8.373,112,4.093,69,4.280,37.9,21.8,22.1,21.3
71251,Patrick Chung,SS,525,168,100,68,35,1.050480,0.030014,17,...,128,3.448,80,2.172,48,1.276,40.5,20.8,17.0,26.5
79848,Malcolm Jenkins,SS,630,101,78,23,15,4.104502,0.273633,11,...,83,3.432,66,2.770,17,0.662,22.8,14.9,14.1,17.4
80668,Captain Munnerlyn,CB,397,129,59,70,28,-12.090908,-0.431818,8,...,97,5.602,49,2.818,48,2.784,54.3,21.7,13.6,28.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2560755,Josh Jackson,CB,397,199,143,56,37,4.856522,0.131257,26,...,158,5.033,114,4.080,44,0.953,28.1,18.6,18.2,19.6
2560916,Denzel Ward,CB,469,185,163,22,41,-16.524718,-0.403042,36,...,142,6.755,125,5.997,17,0.758,11.9,22.2,22.1,22.7
2560952,Jaire Alexander,CB,403,170,115,55,49,-1.467478,-0.029949,33,...,115,6.185,78,4.585,37,1.600,32.4,28.8,28.7,29.1
2561001,Grant Haley,CB,245,124,51,73,25,11.187032,0.447481,10,...,95,4.345,40,1.607,55,2.738,58.9,20.2,19.6,20.5


In [22]:
man_df['position'].value_counts()

CB    81
SS     2
FS     1
S      1
Name: position, dtype: int64

In [23]:
man_df = df[df['position']=='CB']
man_df

,displayName,position,snaps,man-total,man,man-over,targeted man coverage count,man delta epa total,man delta epa rate,targeted man-iso coverage count,...,non targets,non targets epa,non targets - iso,non targets epa - iso,non targets - over,non targets epa - over,safety help %,man target %,man-iso target %,man-over target %
1302,Aqib Talib,CB,202,44,25,19,10,-1.184709,-0.118471,7,...,33,1.225,18,0.792,15,0.433,43.2,22.7,28.0,15.8
2307,Orlando Scandrick,CB,462,246,146,100,60,8.473536,0.141226,38,...,180,5.947,106,3.285,74,2.662,40.7,24.4,26.0,22.0
4365,Brandon Carr,CB,517,248,154,94,54,-3.716949,-0.068832,34,...,181,8.373,112,4.093,69,4.280,37.9,21.8,22.1,21.3
80668,Captain Munnerlyn,CB,397,129,59,70,28,-12.090908,-0.431818,8,...,97,5.602,49,2.818,48,2.784,54.3,21.7,13.6,28.6
89756,Jason McCourty,CB,504,256,156,100,55,-16.882621,-0.306957,30,...,191,5.291,118,3.117,73,2.174,39.1,21.5,19.2,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2560795,Avonte Maddox,CB,343,60,49,11,8,2.303399,0.287925,6,...,49,2.426,40,1.756,9,0.670,18.3,13.3,12.2,18.2
2560916,Denzel Ward,CB,469,185,163,22,41,-16.524718,-0.403042,36,...,142,6.755,125,5.997,17,0.758,11.9,22.2,22.1,22.7
2560952,Jaire Alexander,CB,403,170,115,55,49,-1.467478,-0.029949,33,...,115,6.185,78,4.585,37,1.600,32.4,28.8,28.7,29.1
2561001,Grant Haley,CB,245,124,51,73,25,11.187032,0.447481,10,...,95,4.345,40,1.607,55,2.738,58.9,20.2,19.6,20.5


In [30]:
score(man_df, 'man delta epa rate', 'Man Coverage Score')
score(man_df, 'man-iso delta epa rate', 'Man Iso Score')
score(man_df, 'man-over delta epa rate', 'Man Over Score')

score(man_df, 'safety help %', 'Safety Help % Score')
score(man_df, 'man target %', 'Man Target % Score')
score(man_df, 'non targets epa', 'Man Non-Target EPA Prevented Score', negative=False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
man_df.sort_values('Man Coverage Score',ascending=False)

,displayName,position,snaps,man-total,man,man-over,targeted man coverage count,man delta epa total,man delta epa rate,targeted man-iso coverage count,...,safety help %,man target %,man-iso target %,man-over target %,Man Coverage Score,Man Iso Score,Man Over Score,Safety Help % Score,Man Target % Score,Man Non-Target EPA Prevented Score
2556559,Mike Hilton,CB,375,119,47,72,23,-14.941425,-0.649627,8,...,60.5,19.3,17.0,20.8,70.4,64.6,70.4,33.5,52.9,45.9
2539237,D.J. Hayden,CB,257,93,47,46,22,-12.187035,-0.553956,11,...,49.5,23.7,23.4,23.9,67.3,76.6,67.3,41.4,43.2,38.6
2555177,Mackensie Alexander,CB,295,115,38,77,27,-13.536454,-0.501350,10,...,67.0,23.5,26.3,22.1,65.5,56.5,65.5,28.9,43.7,42.3
2558067,Marlon Humphrey,CB,437,239,144,95,45,-22.161204,-0.492471,24,...,39.7,18.8,16.7,22.1,65.2,64.7,65.2,48.5,54.0,52.3
2555539,Xavien Howard,CB,404,155,106,49,34,-16.521947,-0.485940,20,...,31.6,21.9,18.9,28.6,65.0,64.4,65.0,54.3,47.2,48.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2506861,Brent Grimes,CB,393,69,43,26,13,7.130459,0.548497,7,...,37.7,18.8,16.3,23.1,30.5,28.9,30.5,49.9,54.0,40.6
2558061,Marshon Lattimore,CB,523,242,151,91,49,27.491064,0.561042,35,...,37.6,20.2,23.2,15.4,30.1,33.8,30.1,50.0,50.9,62.6
2495504,Patrick Peterson,CB,505,166,116,50,19,12.891576,0.678504,16,...,30.1,11.4,13.8,6.0,26.2,42.3,26.2,55.4,70.2,56.3
2560753,Donte Jackson,CB,487,141,106,35,24,17.098799,0.712450,18,...,24.8,17.0,17.0,17.1,25.1,32.9,25.1,59.2,57.9,46.1


In [33]:
man_df[['displayName','position','Man Coverage Score','Man Target % Score','Safety Help % Score','Man Non-Target EPA Prevented Score']]

,displayName,position,Man Coverage Score,Man Target % Score,Safety Help % Score,Man Non-Target EPA Prevented Score
1302,Aqib Talib,CB,52.8,45.4,46.0,35.5
2307,Orlando Scandrick,CB,44.1,41.7,47.8,49.9
4365,Brandon Carr,CB,51.1,47.4,49.8,57.4
80668,Captain Munnerlyn,CB,63.2,47.6,38.0,48.9
89756,Jason McCourty,CB,59.0,48.1,48.9,47.9
...,...,...,...,...,...,...
2560795,Avonte Maddox,CB,39.2,66.1,63.9,39.2
2560916,Denzel Ward,CB,62.2,46.5,68.5,52.4
2560952,Jaire Alexander,CB,49.8,32.0,53.7,50.7
2561001,Grant Haley,CB,33.9,50.9,34.7,45.0


In [26]:
man_iso_df = man_df[man_df['man'] > 100].sort_values('Man Iso Score',ascending=False)

In [27]:
score(man_iso_df, 'man-iso delta epa rate', 'Man Iso Score')

In [28]:
score(man_iso_df, 'safety help %', 'Safety Help % Score')
score(man_iso_df, 'man target %', 'Man Target % Score')
score(man_iso_df, 'man-iso target %', 'Man Iso Target % Score')
score(man_iso_df, 'man-over target %', 'Man Over Target % Score')
score(man_iso_df, 'non targets epa - iso', 'Man Iso Non-Target Coverage Score', negative=False)

In [29]:
man_iso_df.sort_values('Man Iso Score',ascending=False)

,displayName,position,snaps,man-total,man,man-over,targeted man coverage count,man delta epa total,man delta epa rate,targeted man-iso coverage count,...,man-over target %,Man Coverage Score,Man Iso Score,Man Over Score,Safety Help % Score,Man Target % Score,Man Iso Target % Score,Man Over Target % Score,Man Non-Target Coverage Score,Man Iso Non-Target Coverage Score
2558067,Marlon Humphrey,CB,437,239,144,95,45,-22.161204,-0.492471,24,...,22.1,65.2,69.8,65.2,39.5,53.0,57.1,47.7,43.8,43.8
2555539,Xavien Howard,CB,404,155,106,49,34,-16.521947,-0.485940,20,...,28.6,65.0,69.3,65.0,49.9,46.4,52.5,37.6,39.6,42.5
2543681,Kyle Fuller,CB,593,246,151,95,56,-16.636283,-0.297076,33,...,24.2,58.7,62.7,58.7,40.9,44.5,46.1,44.4,46.9,46.1
2539338,B.W. Webb,CB,538,261,149,112,43,-0.508883,-0.011834,21,...,19.6,49.2,62.3,49.2,35.3,57.8,62.6,51.6,57.6,50.1
2533062,Stephon Gilmore,CB,588,356,244,112,66,-26.421065,-0.400319,51,...,13.4,62.1,62.3,62.1,50.1,53.6,48.3,61.3,55.5,53.3
2557867,Tre'Davious White,CB,487,253,176,77,46,-15.854256,-0.344658,30,...,20.8,60.3,61.9,60.3,51.5,54.2,56.5,49.7,61.0,61.5
2559164,Michael Davis,CB,336,172,129,43,41,-18.144583,-0.442551,33,...,18.6,63.5,60.7,63.5,58.5,42.4,38.3,53.1,37.4,40.6
89756,Jason McCourty,CB,504,256,156,100,55,-16.882621,-0.306957,30,...,25.0,59.0,60.3,59.0,40.2,47.3,51.9,43.2,38.9,38.0
2532875,Janoris Jenkins,CB,563,244,159,85,44,-6.186202,-0.140595,26,...,21.2,53.5,60.0,53.5,45.8,54.6,57.8,49.1,63.9,61.4
2557840,Chidobe Awuzie,CB,476,238,183,55,54,-10.185851,-0.188627,43,...,20.0,55.1,59.3,55.1,60.9,44.7,42.8,51.0,62.5,65.0


In [34]:
man_iso_df[['displayName','position','Man Iso Score','Safety Help % Score','Man Iso Target % Score','Man Iso Non-Target Coverage Score']]

,displayName,position,Man Iso Score,Safety Help % Score,Man Iso Target % Score,Man Iso Non-Target Coverage Score
2558067,Marlon Humphrey,CB,69.8,39.5,57.1,43.8
2555539,Xavien Howard,CB,69.3,49.9,52.5,42.5
2543681,Kyle Fuller,CB,62.7,40.9,46.1,46.1
2539338,B.W. Webb,CB,62.3,35.3,62.6,50.1
2533062,Stephon Gilmore,CB,62.3,50.1,48.3,53.3
2557867,Tre'Davious White,CB,61.9,51.5,56.5,61.5
2559164,Michael Davis,CB,60.7,58.5,38.3,40.6
89756,Jason McCourty,CB,60.3,40.2,51.9,38.0
2532875,Janoris Jenkins,CB,60.0,45.8,57.8,61.4
2557840,Chidobe Awuzie,CB,59.3,60.9,42.8,65.0
